In [1]:
import time

import codecs

import sys
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('utf-8')
sys.stdout = stdout

In [2]:
import gensim
import multiprocessing
from konlpy.tag import Mecab
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors

In [3]:
text = []
mecab = Mecab()

for news_name in ['chosun_full', 'donga_full', 'hani_full', 'joongang_full', 'kh_full'] :
    with open('./Data/news/'+ news_name + '.txt', 'r') as f:
        lines = f.read()
        sentences = lines.split('.')
        
        for i in range(len(sentences)):
            dic = mecab.morphs(sentences[i])
            dicSize = len(dic)
                    
            for idx in range(dicSize-1):
                if(idx==dicSize):
                        break
                
                else:
                    if(dic[idx]=='새' and dic[(idx+1)]=='정치') :
                        dic.remove('새')
                        dic.remove('정치')
                        dic.insert(idx,"새정치")
                        dicSize = dicSize-1
            
                text.append(dic)
    print(news_name + "finished")
      

chosun_fullfinished
donga_fullfinished
hani_fullfinished
joongang_fullfinished
kh_fullfinished


In [ ]:
import time

start_time = time.time() 


modelCbow= Word2Vec(text, min_count=15, size = 100, sg=0, iter= 5) #sg=0 : cbow
modelSkip = Word2Vec(text, min_count=15, size = 100, sg=1, iter= 5) #sg=1 : skip gram
print("word2vec success")

vectorsCbow = modelCbow.wv
vectorsSkip = modelSkip.wv
print("model.wv assign success")

vectorsCbow.save('modelCbow.bin')
vectorsSkip.save('modelSkip.bin')
print("model saved success")


print("Elapsed time: %s seconds" %(time.time() - start_time))

# Loading and Result

**loading models**

In [ ]:
word_vectors = KeyedVectors.load('./original/modelCbow.bin')


In [ ]:
a="\xec\xb5\x9c\xec\x88\x9c\xec\x8b\xa4".encode("utf-8")
print a



**word similarity**

In [ ]:
name_list = ["박근혜","오바마","김정은","아베","청와대","백악관","새누리당","최순실"]

for idx in range(len(name_list)):
    name_list[idx] = name_list[idx].encode("utf-8")

similarity_list = []
for name in name_list:
    name = name.encode("utf-8")
    w = word_vectors.most_similar(name, topn=1)
    w1 = w[0][0]
    w2 = w[0][1]
    print "[%s]'s most similar: [%s]" %(name, w1),
    print(w2)
    similarity_list.append("[%s]'s most similar: [%s]" % (name, (w)))
    


**word analogy**

In [ ]:
def word_analogy(w1, w2, w3):
    return word_vectors.most_similar(positive=[w1, w2], negative=[w3], topn=1)

words_list = [["서울","일본","한국"],["서울","미국","한국"],["서울","북한","한국"],
                ["박근혜","일본","한국"],["박근혜","미국","한국"],["박근혜","북한","한국"],
                ["새누리당","미국","한국"],["청와대","미국","한국"],["민주주의","북한","한국"],
                ["김무성","새정치","새누리당"],["보수","새정치","새누리당"],["새정치","미국","한국"]]

for element in words_list:
    for idx in range(len(element)):
        element[idx] = element[idx].encode("utf-8")

d = {"%s+%s-%s"%(words[0],words[1],words[2]):word_analogy(words[0],words[1],words[2])[0][0] for words in words_list}

for key in d:
    print "[%s] corresponds to [%s]" % (key, d[key])
